**Create png for dcm files which contain nodules only**
<br>
**png can be saved in one main folder or seperate folders for each patient**

In [0]:
from bs4 import BeautifulSoup
import os
import numpy as np
import re
import pydicom
import re
import mritopng

In [0]:
#data_path_root = folder containing all patient folders
data_path_root="F:\Lunatest"
#data_path_root_list = list of patient folder name
data_path_root_list=os.listdir(data_path_root)
#png_folder = folder to save png
png_folder="F:\PNG_folder"

In [0]:
#Read xml and return 'roi'
def froi(data_path):
    global data_roi
    os.chdir(data_path)
    for file in os.listdir():
        if file.endswith('.xml'):
            xmlname=file
    data_xml=open(xmlname,'r')
    data_xml_contents=data_xml.read()
    soup=BeautifulSoup(data_xml_contents,'html.parser')
    data_roi=soup.findAll('roi')
    return data_roi

In [0]:
#Get image id and Z position from 'roi' data and create a dict
def fIDZdict():
    global dictIDZpos
    ID=[]
    Zpos=[]
    for i in range(len(data_roi)):
        ID.append(re.findall("<imagesop_uid>(.*)</imagesop_uid>",str(data_roi[i])))
        Zpos.append(re.findall('<imagezposition>(.*)</imagezposition>',str(data_roi[i])))
    dictIDZpos={}
    for i in range(len(Zpos)):
        dictIDZpos[ID[i][0]]=float(Zpos[i][0].strip())
    return dictIDZpos

In [0]:
#Get id from dcm and create dict of id and dcmname
def fIDdcmnamedict():
    global dictIDdcmname
    fileList=[]
    for file in os.listdir(data_path):
        if file.endswith('.dcm'):
            fileList.append(file)
    dcmIDList=[]
    for file in fileList:
        dcminfo=pydicom.dcmread(file)
        dcmID=dcminfo.SOPInstanceUID.strip()
        dcmIDList.append(dcmID)
    dictIDdcmname={}
    for i in range(len(fileList)):
        dictIDdcmname[dcmIDList[i]]=fileList[i]
    return dictIDdcmname

In [0]:
#make a dict of Z position and dcmname and return dcmnames which are to be converted to png
#dcmnames to be converted to png are returned as a list : dcmlist
def fdcmlist():
    global dcmlist
    dcmdict={}
    for keyx in dictIDZpos:
        dcmdict[dictIDZpos[keyx]]=dictIDdcmname[keyx]
    dcmlist=list(dcmdict.values())
    return dcmlist

In [0]:
# ask user if they want to save png in one folder or create child png folders
def ask_for_png_folder():
    import sys
    global ask_var
    print("\nEnter 'e' to terminate program\n")
    print("Enter\n'y' for creating child png folder\n'n' for saving png in parent png folder")
    while True:
        ask_var=input("Enter : ")
        if (ask_var == 'y') or (ask_var == 'n'):
            break
        elif ask_var=='e':
            sys.exit()
        else:
            print('Wrong Input')
            continue
                
    return ask_var

In [0]:
#save png in png_folder or create child folders
def png_to_save_folder(png_folder):
    os.chdir(png_folder)
    if ask_var=='n':
        return png_folder
    elif ask_var=='y':
        pfname='png_'+data_path_root_list[folder][-4:]
        if os.path.exists(pfname)==False:
            os.mkdir(pfname)
        return os.path.join(png_folder,pfname)

In [0]:
#write png to folder specified by png_to_save_folder()
def fwritepng():
    png_path=png_to_save_folder(png_folder)
    for dfile in dcmlist:
        #dfilepathname is the full path name of dcm file
        dfilepathname=os.path.join(data_path,dfile)
        #pfile is the name given to png file
        pfile=data_path_root_list[folder][-4:]+'_'+dfile[3:6]+'.png'
        #pfilepathname is the full path name of png file
        pfilepathname=os.path.join(png_path,pfile)
        mritopng.convert_file(dfilepathname,pfilepathname)

In [0]:
global ask_var,data_roi,dictIDZpos,dictIDdcmname,dcmlist,Zlist,Xlist,Ylist
ask_for_png_folder()
for folder in range(0,5):
#for folder in range(0,len(data_path_root_list)):
    data_path=os.path.join(data_path_root,data_path_root_list[folder])
    print(data_path)
    png_to_save_folder(png_folder)
    froi(data_path)
    pid=data_path[-4:]
    fIDZdict()
    fIDdcmnamedict()
    fdcmlist()
    fwritepng()
print("Task Completed")